In [1]:
from efficient_apriori import apriori
from lxml import etree
import time
import csv
from selenium import webdriver
driver = webdriver.Chrome()

In [2]:
director = u'张艺谋'
file_name = './' + director + '.csv'
base_url = 'https://movie.douban.com/subject_search?search_text='+director+'&cat=1002&start='
out = open(file_name,'w', newline='', encoding='utf-8-sig')
csv_write = csv.writer(out, dialect='excel')
flags=[]

In [ ]:
def download(request_url):
    driver.get(request_url)
    time.sleep(1)
    html = driver.find_element_by_xpath("//*").get_attribute("outerHTML")
    html = etree.HTML(html)
    # 设置电影名称，导演演员 的 XPATH
    movie_lists = html.xpath("/html/body/div[@id='wrapper']/div[@id='root']/div[1]//div[@class='item-root']/div[@class='detail']/div[@class='title']/a[@class='title-text']")
    name_lists = html.xpath("/html/body/div[@id='wrapper']/div[@id='root']/div[1]//div[@class='item-root']/div[@class='detail']/div[@class='meta abstract_2']")
    # 获取返回的数据个数
    num = len(movie_lists)
    if num > 15:  # 第一页会有 16 条数据
        # 默认第一个不是，所以需要去掉
        movie_lists = movie_lists[1:]
        name_lists = name_lists[1:]
    for (movie, name_list) in zip(movie_lists, name_lists):
        # 会存在数据为空的情况
        if name_list.text is None: 
            continue
        # 显示下演员名称
        print(name_list.text)
        names = name_list.text.split('/')
        # 判断导演是否为指定的 director
        if names[0].strip() == director and movie.text not in flags:
            # 将第一个字段设置为电影名称
            names[0] = movie.text
            flags.append(movie.text)
    csv_write.writerow(names)
    print('OK')   # 代表这页数据下载成功
    print(num)
    if num >= 14: # 有可能一页会有 14 个电影
        # 继续下一页
        return True
    else:
        # 没有下一页
        return False

In [ ]:
# 开始的 ID 为 0，每页增加 15
start = 0
while start<10000: # 最多抽取 1 万部电影
    request_url = base_url + str(start)
    # 下载数据，并返回是否有下一页
    flag = download(request_url)
    if flag:
        start = start + 15
    else:
        break
out.close()
print('finished')

In [2]:
lists = csv.reader(open('张艺谋.csv', 'r', encoding='utf-8-sig'))

In [3]:
data = []
for names in lists:
    name_new = []
    for name in names:
        name_new.append(name.strip())
    data.append(name_new[1:])

In [4]:
# data中有空数组
data2 = [x for x in data if x != []]

In [32]:
itemsets, rules= apriori(data2, min_support=0.5, min_confidence=1)
print(itemsets)
print(rules)

{}
[]


可以发现，在最小支持度为0.5时，没有演员。

In [28]:
# 挖掘频繁项集和频繁规则
itemsets, rules = apriori(data2, min_support=0.3,  min_confidence=1)
print(itemsets)
print(rules)


{1: {('巩俐',): 9}}
[]


在支持度为0.3时，巩俐老师入选，在张导29部电影中，出现了9次。

In [51]:
itemsets, rules = apriori(data2, min_support=0.06,  min_confidence=1)
print(itemsets)
print(rules)

{1: {('倪大红',): 3, ('傅彪',): 2, ('刘佩琦',): 2, ('刘德华',): 2, ('姜文',): 2, ('孙红雷',): 3, ('巩俐',): 9, ('李保田',): 3, ('李曼',): 2, ('李雪健',): 5, ('杨凤良',): 2, ('牛犇',): 2, ('章子怡',): 3, ('葛优',): 3, ('赵本山',): 2, ('郭涛',): 2, ('闫妮',): 2, ('陈道明',): 2}, 2: {('倪大红', '巩俐'): 2, ('傅彪', '李雪健'): 2, ('刘佩琦', '巩俐'): 2, ('孙红雷', '赵本山'): 2, ('巩俐', '李保田'): 2, ('巩俐', '杨凤良'): 2, ('巩俐', '葛优'): 2, ('巩俐', '郭涛'): 2, ('李保田', '李雪健'): 2}}
[{傅彪} -> {李雪健}, {刘佩琦} -> {巩俐}, {赵本山} -> {孙红雷}, {杨凤良} -> {巩俐}, {郭涛} -> {巩俐}]


当支持度为0.06时，有关联出现。